# Machine learning and deep learning methods applied to predicting customer status

Purpose: Predict active and not active clients based on the proposed data structure and naive data structure. Here, we implemented Support Vector Machines (SVM), Random Forest (RF), K-nearest Neighbours (KNN) and Lasso.

Author: Gabriel Rodrigues Palma and Rafael de Andrade Moral

# Packages used in the project

In [1]:
# visualisation modules
import matplotlib.pyplot as plt

# Data manipulation modules
import numpy as np
import pandas as pd

# Machine learning modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Deep learning modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import LeakyReLU

# Machine learning packages
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import collections
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Lasso
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import LeaveOneOut

# Additional packages
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, Flatten, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import L1
from keras import metrics
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

# Testing GPU from MacOs
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


# Functions used in the project

In [2]:
def create_dataset(path, response_class):
    ''' This function reads and prepare the datasets for applying the ML and DL methods'''
    data = pd.read_csv(path)
    data = data.drop(columns = ['Unnamed: 0', 'Subject'])
    explanatory_variables = data.drop(columns = response_class)
    response_variables = data[response_class]        
    binarizer = LabelBinarizer()
    response_variables = binarizer.fit_transform(response_variables)
    
    onehot_encoder = OneHotEncoder(sparse=False)
    hot_encode_response_variable = onehot_encoder.fit_transform(np.array(response_variables).reshape(-1, 1))          
    
    return(explanatory_variables, 
           hot_encode_response_variable, response_variables)

def check_zero_division_and_get_rates(cm):
    ''' This functions checks for divisions per zeros in the computation of 
       True and false positive rate based on the confusion matrix array. Also, 
       this function returns the checked rates'''
    
    if any(np.sum(cm, axis = 1)==0):            
            tpr = cm[:,1][1]/(np.sum(cm, axis = 1)[1]+1e-16)
            fpr = cm[:,1][0]/(np.sum(cm, axis = 1)[0]+1e-16)    
    else:           
        rates = cm[:,1]/np.sum(cm, axis = 1)    
        fpr = rates[0]    
        tpr = rates[1]
    if np.isnan(tpr):        
        tpr=0
        
    return(tpr, fpr)

def check_and_compute_rates(predictions, 
                            classes, 
                            cm):    
    ''' This function returns the values of true and false positive rate for special cases
       where the division is not possible to obtain automatically based on the confusion
       matrix array provided by sklearn'''
        
    if (sum(predictions) == 0 and sum(classes) == 0):        
        fpr = 0
        tpr = 0
    elif (sum(predictions) == len(predictions) and sum(classes) == len(classes)):        
        fpr = 0
        tpr = 1
    else:            
        tpr, fpr = check_zero_division_and_get_rates(cm)
        
    return(tpr, fpr)
    
def get_rates(y_pred,
              y_true):
    ''' This function get the true and false positive rates based on the 
       predictied'''
            
    cm = confusion_matrix(y_true = y_true, y_pred = y_pred)
    tpr, fpr = check_and_compute_rates(y_pred, y_true, cm)
        
    return(tpr, fpr)

def get_rates_by_cross_validation(raw_data):
    ''' This function obtains the accuracy, true and false positive rates 
       based on the cross k-fold cross validation'''
            
    for train_index, test_index in KFold(n_splits=5, shuffle=True).split(patterns):
        
        x_train, x_test = patterns[train_index], patterns[test_index]
        y_train, y_test = classes[train_index], classes[test_index]
                
        pbp_predictions = pbp_prediction(patterns_array=x_test, 
                                         clustered_patterns = clustered_patterns, 
                                         d_base = d_base, alpha = alpha, 
                   outbreak_p_means = prediction.obtain_p_means_with_distance,
                   outbreak_prediction = prediction.predict_with_distance)
        rates = get_rates(predictions = pbp_predictions, classes = y_test)
        tpr.append(rates[0])
        fpr.append(rates[1])
    
    return(np.mean(tpr[tpr!=np.nan]), np.mean(fpr[fpr!=np.nan]))

def get_statistics(y_pred, y_true):
    ''' This function obtains the following statistics: Accuracy, True positive rate and 
       False positive rate'''

    auc = roc_auc_score(y_score = y_pred,
              y_true = y_true)
    acc = accuracy_score(y_pred = y_pred,
                         y_true = y_true)
    rates = get_rates(y_pred = y_pred,
              y_true = y_true)
    tpr = rates[0]
    fpr = rates[1]
    return([acc, tpr, fpr, auc])

# def get_methods_performance(X_train, X_test, 
#                             y_train, y_test, 
#                             raw_data):
#     ''' This function obtain the performance of each selected model'''
#     kf = KFold(n_splits = 5, random_state = 42, shuffle = True)
#     # Support Vector Machine -----
#     ## Non-linear
#     nonlinear_svm = svm.NuSVC(gamma="auto")
#     # Create the parameter space
#     params = {"kernel": ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
#               "gamma": ['auto', 'scale'],
#              "degree": np.arange(1, 10, 1)}
#     nonlinear_svm_cv = RandomizedSearchCV(nonlinear_svm, params, cv=kf)
#     nonlinear_svm_cv.fit(X_train, y_train)
#     nonlinear_svm_predictions = nonlinear_svm_cv.predict(X_test)
    
#     ## Polinomial
#     polinomial_svm = svm.SVC(kernel='poly', degree = 3)
#     polinomial_svm.fit(X_train, y_train)
#     polinomial_svm_predictions = polinomial_svm.predict(X_test)
    
#     # Random forest -----
#     rf = RandomForestClassifier(max_depth=5, random_state=0, n_estimators = 1000)
#     rf.fit(X_train, y_train)
#     random_forest_predictions = rf.predict(X_test)
    
#     # KNN -----
#     knn = KNeighborsClassifier(n_neighbors=1)
#     knn.fit(X_train, y_train)
#     knn_predictions = knn.predict(X_test)
    
#     # Lasso -----
#     lasso = Lasso(alpha=0.02)
#     lasso.fit(X_train, y_train)
#     lasso_predictions = np.round(lasso.predict(X_test))
    
#     # Deep Neural Network -----
# #     if raw_data==True:
# #         dnn = keras.models.load_model('DNN/DNN_model_params.h5')
# #         dnn_predictions = np.round(dnn.predict(X_test))
# #     elif raw_data == 'combined':
# #         dnn = keras.models.load_model('DNN/DNN_model_fulldata.h5')
# #         dnn_predictions = np.round(dnn.predict(X_test))
# #     else:    
# #         dnn = keras.models.load_model('DNN/DNN_model_hmm.h5')
# #         dnn_predictions = np.round(dnn.predict(X_test))

    
#     # Obtaining statistics -----
#     polynomial_svm_performance = get_statistics(y_pred = polinomial_svm_predictions,
#                                         y_true = y_test)
#     nonlinear_svm_performance = get_statistics(y_pred = nonlinear_svm_predictions,
#                                        y_true = y_test)
#     random_forest_performance = get_statistics(y_pred = random_forest_predictions,
#                                        y_true = y_test)
#     knn_performance = get_statistics(y_pred = knn_predictions,
#                                        y_true = y_test)    
#     lasso_performance = get_statistics(y_pred = lasso_predictions,
#                                        y_true = y_test)
# #     dnn_performance = get_statistics(y_pred = dnn_predictions,
# #                                        y_true = y_test)

#     return(polynomial_svm_performance, nonlinear_svm_performance, 
#            random_forest_performance, knn_performance, 
#            lasso_performance)
    
# def get_results_data(explanatory_variables,                             
#                      response_variables,                                                         
#                      test_size, raw_data):
#     ''' This function obtain the performance of the ML and DL methods based on
#        the prediction of client status'''
#     X_train, X_test, y_train, y_test = train_test_split(explanatory_variables, 
#                                                         response_variables, 
#                                                         test_size = test_size, 
#                                                         random_state = 42)
   
#     # Obtaining methods performance
#     polynomial_svm_performance, nonlinear_svm_performance, \
#     random_forest_performance, knn_performance, \
#     lasso_performance = get_methods_performance(X_train, X_test, 
#                                               y_train, y_test, 
#                                               raw_data)
        
    
#     model_outputs_performance = pd.DataFrame({'Statistics':['Accuracy', 
#                                                             'True Positive Rate', 
#                                                             'False Positive Rate', 
#                                                             'AUROC'],
#                                               'Polinomial SVM': [polynomial_svm_performance[0], 
#                                                                  polynomial_svm_performance[1], 
#                                                                  polynomial_svm_performance[2],
#                                                                 polynomial_svm_performance[3]], 
#                                               'Non linear SVM': [nonlinear_svm_performance[0], 
#                                                                  nonlinear_svm_performance[1], 
#                                                                  nonlinear_svm_performance[2], 
#                                                                 nonlinear_svm_performance[3]], 
#                                               'Random Forest': [random_forest_performance[0], 
#                                                                 random_forest_performance[1], 
#                                                                 random_forest_performance[2], 
#                                                                random_forest_performance[3]], 
#                                               'KNN': [knn_performance[0], 
#                                                       knn_performance[1], 
#                                                       knn_performance[2], 
#                                                       knn_performance[3]],                                                
#                                               'Lasso': [lasso_performance[0], 
#                                                         lasso_performance[1], 
#                                                         lasso_performance[2], 
#                                                         lasso_performance[3]]})

#     return(model_outputs_performance)
    

In [3]:
def get_methods_predictions(X_train, X_test, 
                            y_train, y_test, 
                            data_type):
    ''' This function obtain the performance of each selected model'''
    kf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    # Support Vector Machine -----
    ## Non-linear
    nonlinear_svm = svm.NuSVC()
    
    nonlinear_svm.fit(X_train, y_train)
    
    nonlinear_svm_predictions = nonlinear_svm.predict(X_test)
    
    ## Polinomial
    polinomial_svm = svm.SVC(kernel='poly', degree = 3)
    polinomial_svm.fit(X_train, y_train)
    polinomial_svm_predictions = polinomial_svm.predict(X_test)
    
    # Random forest -----
    rf = RandomForestClassifier(max_depth=5, random_state=0, n_estimators = 1000)
    rf.fit(X_train, y_train)
    random_forest_predictions = rf.predict(X_test)
    
    # KNN -----
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X_train, y_train)
    knn_predictions = knn.predict(X_test)
    
    # Lasso -----
    lasso = Lasso(alpha=0.02)
    lasso.fit(X_train, y_train)
    lasso_predictions = np.round(lasso.predict(X_test))
    
    # Deep Neural Network -----

    dnn = Sequential()
    dnn.add(Dense(100, input_shape=(4,)))
    dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
    dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
    dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
    dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
    dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
    dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
    dnn.add(Dense(1, activation = 'sigmoid'))
    dnn.compile(optimizer = 'adam', 
                  loss = 'mse', 
                 metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
    dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
    dnn_predictions = np.round(dnn.predict(X_test))            

    return(polinomial_svm_predictions, nonlinear_svm_predictions, 
           random_forest_predictions, knn_predictions, 
           lasso_predictions, dnn_predictions)

def get_methods_predictions_loo(polinomial_svm_predictions, nonlinear_svm_predictions, 
                                random_forest_predictions, knn_predictions, 
                                lasso_predictions, dnn_predictions, 
                                reponse_variable):
    
    '''This function obtain the methods performance based on their predictions for the leave one out cross valition'''
    polynomial_svm_performance = get_statistics(y_pred = polinomial_svm_predictions,
                                    y_true = reponse_variable)
    nonlinear_svm_performance = get_statistics(y_pred = nonlinear_svm_predictions,
                                       y_true = reponse_variable)
    random_forest_performance = get_statistics(y_pred = random_forest_predictions,
                                       y_true = reponse_variable)
    knn_performance = get_statistics(y_pred = knn_predictions,
                                       y_true = reponse_variable)    
    lasso_performance = get_statistics(y_pred = lasso_predictions,
                                       y_true = reponse_variable)
    dnn_performance = get_statistics(y_pred = dnn_predictions,
                                        y_true = reponse_variable)
    return(polynomial_svm_performance, nonlinear_svm_performance, 
           random_forest_performance, knn_performance, 
           lasso_performance, dnn_performance)

    
def get_results_data_loo(explanatory_variables,                         
                         response_variables,                                                         
                         data_type):
    ''' This function obtain the performance of the ML and DL methods based on
       the prediction of client status based on leave one out cross validation'''

    loo = LeaveOneOut()
    loo.get_n_splits(explanatory_variables)
    polinomial_svm_predictions, nonlinear_svm_predictions, \
           random_forest_predictions, knn_predictions, \
           lasso_predictions, dnn_predictions = ([], [], [], [], [], [])
    
    for train_index, test_index in loo.split(explanatory_variables):
        X_train, X_test = explanatory_variables[train_index], explanatory_variables[test_index]
        y_train, y_test = response_variables[train_index], response_variables[test_index]                
             
        ## Correcting errors related to column vector (n_observations, ) to (, n_observations)
        y_train = y_train.ravel() 
        y_test = y_test.ravel()        
    
        polynomial_svm_prediction, nonlinear_svm_prediction, \
        random_forest_prediction, knn_prediction, \
        lasso_prediction, dnn_prediction = get_methods_predictions(X_train, X_test, 
                                                   y_train, y_test, 
                                                   data_type)
        
        polinomial_svm_predictions.append(polynomial_svm_prediction[0])
        nonlinear_svm_predictions.append(nonlinear_svm_prediction[0])
        random_forest_predictions.append(random_forest_prediction[0])
        knn_predictions.append(knn_prediction[0])        
        lasso_predictions.append(lasso_prediction[0]) 
        dnn_predictions.append(dnn_prediction[0])
 
    polynomial_svm_performance, nonlinear_svm_performance, \
       random_forest_performance, knn_performance, \
       lasso_performance, dnn_performance = get_methods_predictions_loo(polinomial_svm_predictions, nonlinear_svm_predictions, 
                            random_forest_predictions, knn_predictions, 
                            lasso_predictions, dnn_predictions, 
                            list(response_variables.flatten()))
    
    model_outputs_performance = pd.DataFrame({'Statistics':['Accuracy', 
                                                            'True Positive Rate', 
                                                            'False Positive Rate', 
                                                            'AUROC'],
                                              'Polinomial SVM': [polynomial_svm_performance[0], 
                                                                 polynomial_svm_performance[1], 
                                                                 polynomial_svm_performance[2],
                                                                polynomial_svm_performance[3]], 
                                              'Non linear SVM': [nonlinear_svm_performance[0], 
                                                                 nonlinear_svm_performance[1], 
                                                                 nonlinear_svm_performance[2], 
                                                                nonlinear_svm_performance[3]], 
                                              'Random Forest': [random_forest_performance[0], 
                                                                random_forest_performance[1], 
                                                                random_forest_performance[2], 
                                                               random_forest_performance[3]], 
                                              'KNN': [knn_performance[0], 
                                                      knn_performance[1], 
                                                      knn_performance[2], 
                                                      knn_performance[3]],                                                
                                              'Lasso': [lasso_performance[0], 
                                                        lasso_performance[1], 
                                                        lasso_performance[2], 
                                                        lasso_performance[3]], 
                                             'DNN': [dnn_performance[0], 
                                                        dnn_performance[1], 
                                                        dnn_performance[2], 
                                                        dnn_performance[3]]})

    return(model_outputs_performance)

# Importing Datasets

Before obtaining the performance of the learning algorithms selected for this paper, we need to import the datasets related to the Hidden Markov Models features and the peak features.

## Coordinate data Trial 1

In [4]:
coordinates_explanatory_variables_t1, \
  coordinates_hot_encode_response_variable_t1, coordinates_response_variable_t1 = create_dataset(path = '../../output_data/New_data/CoordinateDatasets/ZscoredScaledCoordinates_data_Trial1.csv', 
                                                                                        response_class = 'Class')

In [5]:
coordinates_explanatory_variables_t1.head()

,dis_sum,angle_sum,average_speed,idle_time
0,0.239184,-0.028896,-0.164018,-0.499294
1,0.222208,-0.700580,-0.257932,-0.415221
2,0.033748,-1.025618,-0.778047,0.509589
3,0.253317,-0.076674,-0.078018,-0.583368
4,0.154093,-0.734181,-0.460115,-0.163000


## Raw Midline EEG data Trial 12

In [6]:
coordinates_explanatory_variables_t12, \
  coordinates_hot_encode_response_variable_t12, coordinates_response_variable_t12 = create_dataset(path = '../../output_data/New_data/CoordinateDatasets/ZscoredScaledCoordinates_data_Trial12.csv', 
                                                                                        response_class = 'Class')

In [7]:
coordinates_explanatory_variables_t12.head()

,dis_sum,angle_sum,average_speed,idle_time
0,0.683578,0.109160,0.812528,-0.665551
1,0.434896,-0.032082,0.066974,-0.380730
2,-0.028934,-0.094611,0.049350,-0.523141
3,0.702254,-0.037036,0.819298,-0.665551
4,-0.607236,-0.624665,-1.338644,0.616143


# Obtaining learning algorithms performance based on Leave one out cross validation

Now, we are obtaining accuracy, trus and false positive rates for Polinomial and no linear Support Vector Machines, Random Forests, K-neirest neighbors and Lasso learning algorithms.

## Zscored Coordinate data Trial 1

In [8]:
get_results_data_loo(explanatory_variables = np.array(coordinates_explanatory_variables_t1),                                                  
                     response_variables = coordinates_response_variable_t1,                                                                           
                     data_type = 'All')

Metal device set to: Apple M1


2023-04-06 13:46:29.645950: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-06 13:46:29.646231: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-04-06 13:46:30.085673: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-06 13:46:30.612187: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 102ms/step


2023-04-06 13:46:36.683121: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 13:46:37.781470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 61ms/step


...

1/1 [==============================] - 0s 185ms/step


2023-04-06 13:53:17.175116: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 13:53:18.982770: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 165ms/step


2023-04-06 13:53:27.446495: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.510204,0.795918,0.918367,0.775510,0.306122,0.897959
1,True Positive Rate,0.185185,0.703704,0.888889,0.777778,0.481481,0.851852
2,False Positive Rate,0.090909,0.090909,0.045455,0.227273,0.909091,0.045455
3,AUROC,0.547138,0.806397,0.921717,0.775253,0.286195,0.903199


## Zscored Coordinates data Trial 12

In [9]:
get_results_data_loo(explanatory_variables = np.array(coordinates_explanatory_variables_t12),                                                  
                     response_variables = coordinates_response_variable_t12,                                                                           
                     data_type = 'All')

2023-04-06 13:53:29.252601: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 162ms/step


2023-04-06 13:53:37.986729: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 13:53:39.758752: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 152ms/step


2023-04-06 13:53:48.474685: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 13:53:50.092371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-06 14:02:04.610084: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 14:02:15.354649: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 311ms/step


2023-04-06 14:02:16.816007: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 14:02:26.898247: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 295ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.591837,0.816327,0.857143,0.836735,0.775510,0.938776
1,True Positive Rate,0.518519,0.851852,0.888889,0.851852,0.740741,0.925926
2,False Positive Rate,0.318182,0.227273,0.181818,0.181818,0.181818,0.045455
3,AUROC,0.600168,0.812290,0.853535,0.835017,0.779461,0.940236
